# Search 2.0 Data Planning (Searches & Clicks)

## Introduction

Related ticket: https://github.com/orgs/GSA-TTS/projects/60/views/7?pane=issue&itemId=97252640&issue=GSA-TTS%7Cjemison%7C123


In this notebook, I will be wrangling our data samples to see if they are able to answer our metric questions with their output.

**The focus in this notebook will be on searches & clicks.**

## Data

### Libraries
Importing libraries to work with our JSON data:

In [1]:
# If pandas has not been installed yet:
# !pip install pandas

In [2]:
# for general data wrangling tasks
import pandas as pd

# for working with json
import json

### JSON sample data

We have 4 files currently (as of 2025-03-05):
* **v2search_sample_data.json** (focus for this notebook)
* **v2click_sample_data.json** (focus for this notebook)
* v2indexing_sample_data.json
* search_service_sample_data_cleaned.json

The 'v2' indicates that feedback from our [markdown file](https://github.com/GSA-TTS/jemison/blob/main/docs/architecture/services/json-data-structure-drafts.md) has been implemented.

These JSON structures (and as a result, the code below) will change and evolve as our developer teams iterate.

#### Guiding questions:
##### For partners and the public

1. What queries have been run against which domains?
2. What results (served from which query) have been followed or clicked through?

##### For the team

1. What is our click-through rate (overall, per domain)
2. What is our uptime, and other metrics (e.g. RAM usage, DB usage, query performance, etc.)
3. What % of Federal domains make use of Search (monthly, quarterly)

## Search Sample Data

In [3]:
# Open and read the JSON file v2search_sample_data.json:
with open('./data/v2/v2search_sample_data.json', 'r') as file:
      v2_search_sample_data = pd.json_normalize(json.load(file))

# Print the data
v2_search_sample_data

,query_uuid,search_query,domain,search,referrer_url,query_duration,date_of_search,list_of_results
0,4f7ebebe-f2e4-11ef-bc21-df6c2092c112,passport,www.usa.gov,True,https://www.usa.gov,321,2025-02-19,"[{'result_1': 'result_a'}, {'result_2': 'resul..."
1,4f7tftft-f2e4-11ef-bc21-df6c2092c000,test,www.example.gov,True,https://www.example.gov/page,123,2025-02-19,"[{'result_1': 'result_c'}, {'result_2': 'resul..."
2,5d7tftft-f2e4-11ef-bc21-df6c2092c626,test,www.example.gov,True,https://www.example.gov/page,233,2025-02-19,"[{'result_1': 'result_c'}, {'result_2': 'resul..."
3,9d7tttft-f2e4-11ef-bc21-df6c2092c910,apple,www.fruits.gov,True,https://www.fruits.gov/trees,434,2025-02-20,"[{'result_1': 'result_e'}, {'result_2': 'resul..."
4,2d7tttft-f2e4-11ef-bc21-df6c2092c643,peach,www.fruits.gov,True,https://www.fruits.gov/trees,626,2025-02-20,"[{'result_1': 'result_e'}, {'result_2': 'resul..."
5,7d7sasft-f2e4-11ef-bc21-df6c2092c699,toast,www.bakery.gov,True,http://www.bakery.gov,324,2025-02-21,"[{'result_1': 'result_e'}, {'result_2': 'resul..."
6,1d7sasft-f2e4-11ef-bc21-df6c2092c323,bread,www.bakery.gov,True,http://www.bakery.gov/oven,234,2025-02-21,"[{'result_1': 'result_e'}, {'result_2': 'resul..."


### Search Data Fields

| data\_field | data\_type | category | description | related\_requested\_metric | related\_metric\_question | priority | notes\_questions |
| :---- | :---- | :---- | :---- | :---- | :---- | :---- | :---- |
| **domain** | String | Search / Customer Info | What domain is the user searching on? | CTR (per domain) | What queries have been run against which domains? What % of Federal domains make use of Search (monthly, quarterly) | High | Unique identifier in the new system. In legacy, the unique identifier was site handle / ID. These fields will still exist to handle legacy. Data will be "longer" rather than "wider". |
| **query\_uuid** | String | Click / Search | Connects our search data to click data | CTR; \# of searches; \# of clicks |  | High | result\_data is a new service. It provides the connecting glue between resultsapi (which will store a UUID for each query result) and the links that get clicked (which will need to ping an API endpoint with the UUID and any metadata we want to log about the query). |
| **search\_query** | String | Search | What users search |  |  | High | PII considerations. [Regex](https://docs.google.com/document/d/1Krks4cjzst1_C36ii43X5RR0YvTPWWoEYirGaR4KLH0/edit?usp=sharing) can be applied to the most common patterns, but some information may slip past. |
| **Search** | True/False | Search | Was a search performed? Sum of which will provide \# of searches | CTR; \# of searches |  | High |  |
| **referrer\_url** | URL/String | Search | Page users were on when they ran query |  |  | High | Unsure what information will make it through the Cloud.gov stack, and we want to be careful of PII/user identification when we track down to this level. Note: Would it be enough to remove all tracking elements of the referrer? (everything after '?' in the URL) Users have noted they find it important to be able to filter reports / queries by the referrer URL. |
| **query\_duration** | Integer | Click / Search & Search service info | For searches, how long did it take to load results? |  |  | Medium | Millseconds |
| **date\_of\_search** | Date | Search | Date of search |  |  | High | For aggregation & reporting: At least the granularity of daily. May not need more granularity than this due to space & practicality. Additionally, having date with time may lead to privacy concerns. |
| **list\_of\_results** | JSON | Search | List of search results served |  | What results did users see when they performed a search? What results (served from which query) have been followed or clicked through? | Medium |  |



## Click Sample Data

In [4]:
# Open and read the JSON file v2click_sample_data.json:
with open('./data/v2/v2click_sample_data.json', 'r') as file:
    v2click_sample_data = pd.json_normalize(json.load(file))

# Print the data
v2click_sample_data

,query_uuid,click,date_of_click,click_url,click_url_hostname,click_page_position,device_category,search_type,site_id
0,4f7ebebe-f2e4-11ef-bc21-df6c2092c112,True,2025-02-19,https://www.usa.gov/passport,www.usa.gov,1.0,Desktop,regular,9114
1,4f7tftft-f2e4-11ef-bc21-df6c2092c000,True,2025-02-19,https://www.example.gov/test-page,www.example.gov,4.0,Mobile,regular,9999
2,5d7tftft-f2e4-11ef-bc21-df6c2092c626,False,2025-02-19,None,None,NaN,Mobile,regular,9999
3,9d7tttft-f2e4-11ef-bc21-df6c2092c910,False,2025-02-20,None,None,NaN,Desktop,regular,9996
4,2d7tttft-f2e4-11ef-bc21-df6c2092c643,False,2025-02-20,None,None,NaN,Mobile,regular,9996
5,7d7sasft-f2e4-11ef-bc21-df6c2092c699,True,2025-02-21,http://www.bakery.gov/toaster,www.bakery.gov,1.0,Desktop,regular,9990
6,1d7sasft-f2e4-11ef-bc21-df6c2092c323,False,2025-02-21,None,None,NaN,Mobile,regular,9990


### Click data fields

| data\_field | data\_type | category | description | related\_requested\_metric | related\_metric\_question | priority | notes\_questions |
| :---- | :---- | :---- | :---- | :---- | :---- | :---- | :---- |
| **query\_uuid** | String | Click / Search | Connects our search data to click data | CTR; \# of searches; \# of clicks |  | High | result\_data is a new service. It provides the connecting glue between resultsapi (which will store a UUID for each query result) and the links that get clicked (which will need to ping an API endpoint with the UUID and any metadata we want to log about the query). |
| **click** | True/False | Click / Search | Was a click performed (from the search)? Sum of which will provide \# of clicks from result pages, tied click URL | CTR; \# of clicks |  | High |  |
| **date\_of\_click** | Date | Click | Date of click | \# of clicks |  | High | For aggregation & reporting: At least the granularity of daily. May not need more granularity than this due to space & practicality. Additionally, having date with time may lead to privacy concerns. |
| **click\_url** | URL/String | Click / Search | Page users clicked on from results pages |  | What results (served from which query) have been followed or clicked through? | High |  |
| **click\_url\_hostname** | URL/String | Click / Search | Hostname of page users clicked on from results pages |  |  | Medium | Enables users to filter quickly where traffic is being sent to. |
| **click\_page\_position** | Integer | Click / Search | Which \# result was it on the results page? |  |  | Low |  |
| **device\_category** | String | Click / Search | Mobile, Desktop, Tablet |  |  | Medium |  |
| **search\_type** | String | Click / Search | Regular / best bets |  |  |  | for future use: if we implement something like best bets, this column could be used to hold this information. |
| **site\_id** | Integer | Customer Info | (legacy) Tied with Site Handle |  |  | High | Domain will be the unique identifier in the new system. site handle / ID. These fields will still exist to handle legacy. This will be used to join with other tables to get fields such as: Display Name, Site Handle, Agency |



## Joining our Search and Click data:

In [5]:
v2click_sample_data
v2_search_sample_data

search_click_joined = pd.merge(v2_search_sample_data,
                              v2click_sample_data,
                              how="left",
                              on=["query_uuid"])

search_click_joined

,query_uuid,search_query,domain,search,referrer_url,query_duration,date_of_search,list_of_results,click,date_of_click,click_url,click_url_hostname,click_page_position,device_category,search_type,site_id
0,4f7ebebe-f2e4-11ef-bc21-df6c2092c112,passport,www.usa.gov,True,https://www.usa.gov,321,2025-02-19,"[{'result_1': 'result_a'}, {'result_2': 'resul...",True,2025-02-19,https://www.usa.gov/passport,www.usa.gov,1.0,Desktop,regular,9114
1,4f7tftft-f2e4-11ef-bc21-df6c2092c000,test,www.example.gov,True,https://www.example.gov/page,123,2025-02-19,"[{'result_1': 'result_c'}, {'result_2': 'resul...",True,2025-02-19,https://www.example.gov/test-page,www.example.gov,4.0,Mobile,regular,9999
2,5d7tftft-f2e4-11ef-bc21-df6c2092c626,test,www.example.gov,True,https://www.example.gov/page,233,2025-02-19,"[{'result_1': 'result_c'}, {'result_2': 'resul...",False,2025-02-19,None,None,NaN,Mobile,regular,9999
3,9d7tttft-f2e4-11ef-bc21-df6c2092c910,apple,www.fruits.gov,True,https://www.fruits.gov/trees,434,2025-02-20,"[{'result_1': 'result_e'}, {'result_2': 'resul...",False,2025-02-20,None,None,NaN,Desktop,regular,9996
4,2d7tttft-f2e4-11ef-bc21-df6c2092c643,peach,www.fruits.gov,True,https://www.fruits.gov/trees,626,2025-02-20,"[{'result_1': 'result_e'}, {'result_2': 'resul...",False,2025-02-20,None,None,NaN,Mobile,regular,9996
5,7d7sasft-f2e4-11ef-bc21-df6c2092c699,toast,www.bakery.gov,True,http://www.bakery.gov,324,2025-02-21,"[{'result_1': 'result_e'}, {'result_2': 'resul...",True,2025-02-21,http://www.bakery.gov/toaster,www.bakery.gov,1.0,Desktop,regular,9990
6,1d7sasft-f2e4-11ef-bc21-df6c2092c323,bread,www.bakery.gov,True,http://www.bakery.gov/oven,234,2025-02-21,"[{'result_1': 'result_e'}, {'result_2': 'resul...",False,2025-02-21,None,None,NaN,Mobile,regular,9990


In [6]:
# getting a look at our datatypes
search_click_joined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   query_uuid           7 non-null      object 
 1   search_query         7 non-null      object 
 2   domain               7 non-null      object 
 3   search               7 non-null      bool   
 4   referrer_url         7 non-null      object 
 5   query_duration       7 non-null      int64  
 6   date_of_search       7 non-null      object 
 7   list_of_results      7 non-null      object 
 8   click                7 non-null      bool   
 9   date_of_click        7 non-null      object 
 10  click_url            3 non-null      object 
 11  click_url_hostname   3 non-null      object 
 12  click_page_position  3 non-null      float64
 13  device_category      7 non-null      object 
 14  search_type          7 non-null      object 
 15  site_id              7 non-null      int64  

## Nightly aggregation?

Due to the huge volumes of data expected to come through Search.gov, we will likely need to do a nightly process for aggregation.

Some fields would be summed:
* `search`
* `click`

Other fields, we will take the mean:
* `query_duration`
* `click_page_position`

We may no longer need:
* `query_uuid`
* `list_of_results` (?) <- **Open to input.**

In [7]:
# dictionary of old:new names:
new_agg_names = {'search':'sum_searches',
                'click':'sum_clicks',
                'query_duration':'average_query_duration',
                 'click_page_position':'average_click_pg_pos'
                }

search_click_joined.groupby([
    # 'query_uuid',
    'search_query',
    'domain',
    'referrer_url',
    'date_of_search',
    #'list_of_results',
    'date_of_click',
    'click_url',
    'click_url_hostname',
    'device_category',
    'search_type',
    'site_id'
],as_index=False, dropna=False).agg({ # we need dropna = FALSE or else not all rows will be aggregated in our sums.
    'search':'sum',
    'click':'sum',
    'query_duration':'mean',
    'click_page_position':'mean'}).rename(columns = new_agg_names)

,search_query,domain,referrer_url,date_of_search,date_of_click,click_url,click_url_hostname,device_category,search_type,site_id,sum_searches,sum_clicks,average_query_duration,average_click_pg_pos
0,apple,www.fruits.gov,https://www.fruits.gov/trees,2025-02-20,2025-02-20,NaN,NaN,Desktop,regular,9996,1,0,434.0,NaN
1,bread,www.bakery.gov,http://www.bakery.gov/oven,2025-02-21,2025-02-21,NaN,NaN,Mobile,regular,9990,1,0,234.0,NaN
2,passport,www.usa.gov,https://www.usa.gov,2025-02-19,2025-02-19,https://www.usa.gov/passport,www.usa.gov,Desktop,regular,9114,1,1,321.0,1.0
3,peach,www.fruits.gov,https://www.fruits.gov/trees,2025-02-20,2025-02-20,NaN,NaN,Mobile,regular,9996,1,0,626.0,NaN
4,test,www.example.gov,https://www.example.gov/page,2025-02-19,2025-02-19,https://www.example.gov/test-page,www.example.gov,Mobile,regular,9999,1,1,123.0,4.0
5,test,www.example.gov,https://www.example.gov/page,2025-02-19,2025-02-19,NaN,NaN,Mobile,regular,9999,1,0,233.0,NaN
6,toast,www.bakery.gov,http://www.bakery.gov,2025-02-21,2025-02-21,http://www.bakery.gov/toaster,www.bakery.gov,Desktop,regular,9990,1,1,324.0,1.0


You may notice little has changed about the dataset, other than the two columns being dropped and the column names for our aggregations changing.

This is expected due to our small sample / dummy data.

At much larger volumes, we may see a dramatic reduction as redundant rows get summed / averaged.

## Getting basic metrics

### Total Searches:

In [8]:
total_searches = search_click_joined['search'].sum()

"The total number of searches is: " + str(total_searches)

'The total number of searches is: 7'

In [9]:
# searches by domain

# subset domain and searches columns, then group by, and sum:
total_searches_by_domain = search_click_joined[['domain','search']].groupby(['domain']).sum()

total_searches_by_domain

,search
domain,
www.bakery.gov,2
www.example.gov,2
www.fruits.gov,2
www.usa.gov,1


### Total Clicks:

In [10]:
total_clicks = search_click_joined['click'].sum()

"The total number of clicks is: " + str(total_clicks)

'The total number of clicks is: 3'

In [11]:
# clicks by domain

# subset domain and click columns, then group by, and sum:
total_clicks_by_domain = search_click_joined[['domain','click']].groupby(['domain']).sum()

total_clicks_by_domain

,click
domain,
www.bakery.gov,1
www.example.gov,1
www.fruits.gov,0
www.usa.gov,1


### CTR:
Calculated by dividing: clicks / searches.

In [12]:
# overall CTR for entire dataset:
ctr_calc = search_click_joined['click'].sum() / search_click_joined['search'].sum()

# round to the 4th decimal place
print(ctr_calc.round(4))

0.4286


### CTR by domain:

In [13]:
# columns we would like to focus on for this metric:
cols = ['domain','search','click']

# with our subset, what are the total searches for each domain?
search_click_domain_ctr = search_click_joined[cols].groupby(['domain']).sum()

# calculating our CTR:
search_click_domain_ctr['ctr_by_domain'] = search_click_domain_ctr['click']/search_click_domain_ctr['search']

search_click_domain_ctr.sort_values(by='ctr_by_domain',ascending=False)

,search,click,ctr_by_domain
domain,,,
www.usa.gov,1,1,1.0
www.bakery.gov,2,1,0.5
www.example.gov,2,1,0.5
www.fruits.gov,2,0,0.0


### What queries have been run against which domains?

In [14]:
# columns we would like to focus on for this metric:
cols = ['domain','search_query','search','click']

search_click_joined[cols]

,domain,search_query,search,click
0,www.usa.gov,passport,True,True
1,www.example.gov,test,True,True
2,www.example.gov,test,True,False
3,www.fruits.gov,apple,True,False
4,www.fruits.gov,peach,True,False
5,www.bakery.gov,toast,True,True
6,www.bakery.gov,bread,True,False


Filter our data by a domain:

In [15]:
# with our subset, what are the searches for each domain?
# cols = ['domain','search_query','search','click']
search_click_domain_query = search_click_joined[cols].groupby(['domain','search_query']).sum().sort_values(by='search',ascending=False)

# reset index from group by
search_click_domain_query = search_click_domain_query.reset_index()

# filtering by domain
search_click_domain_query[search_click_domain_query['domain'] == 'www.example.gov']

,domain,search_query,search,click
0,www.example.gov,test,2,1


### What results (served from which query) have been followed or clicked through?

In [16]:
# columns we would like to focus on for this metric:
cols = ['domain','search_query','referrer_url','search','click','click_url']

search_click_joined[cols]

,domain,search_query,referrer_url,search,click,click_url
0,www.usa.gov,passport,https://www.usa.gov,True,True,https://www.usa.gov/passport
1,www.example.gov,test,https://www.example.gov/page,True,True,https://www.example.gov/test-page
2,www.example.gov,test,https://www.example.gov/page,True,False,None
3,www.fruits.gov,apple,https://www.fruits.gov/trees,True,False,None
4,www.fruits.gov,peach,https://www.fruits.gov/trees,True,False,None
5,www.bakery.gov,toast,http://www.bakery.gov,True,True,http://www.bakery.gov/toaster
6,www.bakery.gov,bread,http://www.bakery.gov/oven,True,False,None


In [17]:
# we will group by domain, referrer_url, and search_query:
# cols = ['domain','search_query','referrer_url','search','click','click_url']
search_click_results_followed = search_click_joined[cols].groupby(['domain','referrer_url','search_query']).sum().sort_values(by='click',ascending=False)

# reset index from group by
search_click_results_followed = search_click_results_followed.reset_index()

# search_click_results_followed

# calculating our CTR:
search_click_results_followed['ctr'] = search_click_results_followed['click']/search_click_results_followed['search']

search_click_results_followed.sort_values(by= 'ctr', ascending = False)

,domain,referrer_url,search_query,search,click,click_url,ctr
0,www.bakery.gov,http://www.bakery.gov,toast,1,1,http://www.bakery.gov/toaster,1.0
2,www.usa.gov,https://www.usa.gov,passport,1,1,https://www.usa.gov/passport,1.0
1,www.example.gov,https://www.example.gov/page,test,2,1,https://www.example.gov/test-page,0.5
3,www.bakery.gov,http://www.bakery.gov/oven,bread,1,0,0,0.0
4,www.fruits.gov,https://www.fruits.gov/trees,apple,1,0,0,0.0
5,www.fruits.gov,https://www.fruits.gov/trees,peach,1,0,0,0.0


### What % of Federal domains make use of Search (monthly, quarterly)?

What is the full list of federal domains? This depends on what list is used.

Recommended list:
* https://github.com/cisagov/dotgov-data
    * Note: Many are internal.

To calculate, this would be # using Search.gov / CISA list.

#### Filter by dates, if desired:

If this method is used, be sure to replace all references below to `search_click_joined` with `search_click_subset_date_data`.

In [18]:
# converting column to datetime format:
search_click_joined['date_of_search'] = pd.to_datetime(search_click_joined['date_of_search'])

# subsetting our data:
start_date = '2025-02-19' # inclusive
end_date = '2025-02-20' # inclusive
search_click_subset_date_data = search_click_joined[(search_click_joined['date_of_search'] >= start_date) & (search_click_joined['date_of_search'] <= end_date)]

#### Count of the unique domains, in our data:

In [19]:
# count of the unique domains:

len(search_click_joined['domain'].unique())

4

In [20]:
# divide this over the # of federal domains.
# As of today, this seems to be 1355.

# CISA Github link:
url = 'https://raw.githubusercontent.com/cisagov/dotgov-data/refs/heads/main/current-federal.csv'

cisa_fed_domains = pd.read_csv(url)

cisa_fed_domains.head()

,Domain name,Domain type,Agency,Organization name,City,State,Security contact email
0,acus.gov,Federal - Executive,Administrative Conference of the United States,Administrative Conference of the United States,Washington,DC,info@acus.gov
1,achp.gov,Federal - Executive,Advisory Council on Historic Preservation,Advisory Council on Historic Preservation,Washington,DC,domainsecurity@achp.gov
2,preserveamerica.gov,Federal - Executive,Advisory Council on Historic Preservation,Advisory Council on Historic Preservation,Washington,DC,domainsecurity@achp.gov
3,abmc.gov,Federal - Executive,American Battle Monuments Commission,American Battle Monuments Commission,Arlington,VA,itsec@abmc.gov
4,amtrakoig.gov,Federal - Executive,AMTRAK,Office of Inspector General,Washington,DC,(blank)


#### Count of unique domains in the CISA federal domains list:

In [21]:
# count of unique domains in the CISA federal domains list:
len(cisa_fed_domains["Domain name"].unique())

1355

#### If desired, can do an inner join with CISA list:

Optionally, I will do an inner join to see what domains are in both the CISA data and our sample search data.

For us, this should only be the length of 1, because the other URLs do not exist.

In [22]:
# rename so our join merge will work:
cisa_fed_domains.rename(columns={'Domain name':'domain'}, inplace=True)

# remove the www. from the front of our domains
search_click_joined_no_www = search_click_joined['domain'].str.replace(r'https?://|www\.', '', regex=True)

domains_merge = pd.merge(search_click_joined_no_www,
                              cisa_fed_domains,
                              how="inner")
domains_merge

,domain,Domain type,Agency,Organization name,City,State,Security contact email
0,usa.gov,Federal - Executive,General Services Administration,"GSA,FAS,Technology Transformation Service",Washington,DC,gsa-vulnerability-reports@gsa.gov
